In [2]:
import gym
import numpy as np
import random

from gym.spaces import Box

from metadrive.envs.metadrive_env import MetaDriveEnv
from clip.clip_rewarded_sac import CLIPRewardedSAC
from config import CONFIGS

from stable_baselines3 import PPO

CONFIG = CONFIGS["1"]

In [ ]:
train_env = MetaDriveEnv()

# model = CLIPRewardedSAC(env=env, config=CONFIG)
# Start with ppo to make sure the observation space is correct
model = PPO("MlpPolicy", train_env, n_steps=4096, verbose=1)
model.learn(total_timesteps=1000, log_interval=4)
train_env.close()

In [ ]:
env = MetaDriveEnv(config={"use_render": True})
total_reward = 0
obs, _ = env.reset()

agent = env.agent

try:
    for i in range(1000):
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, done, _, info = env.step(action)
        total_reward += reward
        if done:
            print("episode_reward", total_reward)
            break
finally:
    env.close()

In [ ]:
from metadrive.component.road_network import Road
from waypoints import *

from panda3d.core import LVector3f

env = MetaDriveEnv(config=dict(
    num_scenarios=1,
    start_seed=random.randint(0, 10000),
    random_lane_width=True,
    random_agent_model=True,
    random_lane_num=True,
    use_render=True,
))

try:
    obs, info = env.reset()
    agent = env.agent

    waypoints = get_waypoints(agent)

    point_drawer = env.engine.make_point_drawer(scale=5)
    point_drawer.reset()
    point_drawer.draw_points(waypoints, [[1, 0, 0, 1] for _ in waypoints])
    for i in range(1000):
        obs, reward, terminated, truncated, info = env.step(env.action_space.sample())
        if i % 100 == 0:
            waypoints = get_next_relative_waypoints(agent)
            print(waypoints[0])
finally:
    env.close()

[INFO] Environment: MetaDriveEnv
[INFO] MetaDrive version: 0.4.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector(), main_camera: MainCamera(1200, 900), dashboard: DashBoard()]
[INFO] Render Mode: onscreen
[INFO] Horizon (Max steps per agent): 1000
[INFO] Assets version: 0.4.3
[INFO] Known Pipes: CocoaGraphicsPipe
[WARNING] Since your screen is too small (1470, 956), we resize the window to (1147, 860). (engine_core.py:234)
[INFO] Start Scenario Index: 6019, Num Scenarios : 1


(np.float64(10.0), np.float64(2.183286099714123), 0.5)
(np.float64(10.008096731833472), np.float64(2.141840348629557), 0.5)


SystemExit: 

/opt/anaconda3/envs/metadrive/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3678: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
